In [181]:
import pandas as pd
import numpy as np

In [182]:
# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

In [183]:
# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'country', 'rgdpna', 'rkna', 'emp','avh', 'labsh','rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

In [184]:
# Calculate additional columns
data['L'] = data['emp'] * data['avh']  # L
data['ln_y'] = np.log(data['rgdpna'] / data['L']) # Y/L
data['ln_A'] = np.log(data['rtfpna']) # A
data['ln_k'] = np.log(data['rkna'] / data['L'])  # K/L
data['alpha'] = 1 - data['labsh']  # alpha
data['alpha_x_ln_k'] = data['alpha'] * data['ln_k']  # alpha x (K/L)

In [185]:
# Order by year
data = data.sort_values('year')

In [186]:
# Group by isocode
grouped_data = data.groupby('country')

In [187]:
# Calculate growth rates and Solow residual
data['g_rate'] = (grouped_data['ln_y'].diff() * 100)  # Growth rate of GDP 
data['tfp_g'] = (grouped_data['ln_A'].diff() * 100)  # TFP Growth
data['cap_d'] = (grouped_data['alpha_x_ln_k'].diff() * 100)  # Capital Deepening
data['tfp_s'] = data['tfp_g'] / data['g_rate']  # TFP Share
data['cap_s'] = data['cap_d'] / data['g_rate']  # Capital Share
data['cap_s_2'] = 1 - data['tfp_s'] # Capital Share 2

In [188]:
# Remove missing values
data = data.dropna()

In [189]:
# Calculate summary statistics
summary = data.groupby('country').agg({'g_rate': 'mean','tfp_g': 'mean','cap_d': 'mean','tfp_s': 'mean','cap_s': 'mean', 'cap_s_2': 'mean'})

In [190]:
# Calculate additional summary statistics
summary['Growth Rate'] = summary['g_rate']
summary['TFP Growth'] = summary['tfp_g']
summary['Capital Deepening'] = summary['cap_d']
summary['TFP Share'] = summary['tfp_s']
summary['Capital Share'] = summary['cap_s']
summary['Capital Share 2'] = summary['cap_s_2']

In [191]:
# Print output
print(summary.iloc[:, -6:])

                Growth Rate  TFP Growth  Capital Deepening  TFP Share  \
country                                                                 
Canada             1.117384    0.151360          -0.560312   0.504413   
France             1.087678    0.093816           0.840665   0.315030   
Germany            1.073874    0.358568          -0.336012   0.367890   
Italy              0.285631   -0.772207           0.541721   0.508958   
Japan              1.221666    0.142913          -0.897175  -1.074495   
United Kingdom     1.149943    0.230807           2.748162   0.730544   
United States      1.658237    0.675462           0.303074   0.574376   

                Capital Share  Capital Share 2  
country                                         
Canada               1.252000         0.495587  
France               1.480154         0.684970  
Germany              3.970329         0.632110  
Italy               -8.736051         0.491042  
Japan               32.320018         2.074495  

In [192]:
# Equal to 1?
summary['TFP Growth + Capital Deepening'] = summary['TFP Growth'] + summary['Capital Deepening']
summary['TFP Share + Capital Share'] = summary['TFP Share'] + summary['Capital Share']
summary['TFP Share + Capital Share 2'] = summary['TFP Share'] + summary['Capital Share 2']

In [193]:
print(summary.iloc[:, -3:])

                TFP Growth + Capital Deepening  TFP Share + Capital Share  \
country                                                                     
Canada                               -0.408952                   1.756412   
France                                0.934481                   1.795185   
Germany                               0.022557                   4.338220   
Italy                                -0.230486                  -8.227092   
Japan                                -0.754262                  31.245523   
United Kingdom                        2.978969                   0.674354   
United States                         0.978536                  -4.730059   

                TFP Share + Capital Share 2  
country                                      
Canada                                  1.0  
France                                  1.0  
Germany                                 1.0  
Italy                                   1.0  
Japan                         